In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import torch.optim as optim

import torchvision
from torchvision import datasets as dsets
from torchvision import transforms

from matplotlib.pyplot import imshow
%matplotlib inline

import visdom
import math
import vgg
import numpy as np

In [2]:
vis = visdom.Visdom()
vis.close(env="main")

def plt_tracker(loss_plot, loss_value, num):
    vis.line(X=np.stack(np.arange(num, num+1)),
             Y=np.stack([loss_value]),
             win=loss_plot,
             update='append'
             )

In [3]:
if (torch.cuda.is_available() == 1):
    print("cuda is available")
    device = 'cuda'
else:
    device = 'cpu'

cuda is available


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [6]:
train = dsets.CIFAR10(root='./cifar10/train',transform=transform,train=True,download=True)
test = dsets.CIFAR10(root='./cifar10/test',transform=transform,train=False,download=True)

#check_img=train.__getitem__(0)
#ToPIL=transforms.ToPILImage()
#check_img = ToPIL(check_img[0])
#imshow(check_img)

train_loader=data.DataLoader(train, batch_size=8, shuffle=True, num_workers=4)
test_loader=data.DataLoader(test, batch_size=8, shuffle=False, num_workers=4)

Files already downloaded and verified


Files already downloaded and verified


In [8]:
conv = vgg.make_layers(vgg.cfg['cifar10'],True)
print(conv)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace)
  (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(64,

In [9]:
class tiny_VGG(nn.Module):
    def __init__(self,conv_layer,num_of_class=200, init_weights=True):
        super(tiny_VGG, self).__init__()
        self.conv = conv_layer
        self.fc = nn.Sequential(
            nn.Linear(8*8*64, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, num_of_class))
        
        if init_weights:
            self._initialize_weights()
        
    def forward(self,x):
        x = self.conv(x)
        #print(x.shape)
        x=x.view(x.shape[0], -1)
        
        x=self.fc(x)
        return x
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [17]:
CNN = tiny_VGG(conv,num_of_class=10)
CNN=CNN.to(device)

In [18]:
test_input = torch.Tensor(3,3,32,32).to(device)
out_conv = conv(test_input)
print(out_conv.shape)
out = CNN(test_input)
print(out.shape)

torch.Size([3, 64, 8, 8])
torch.Size([3, 10])
torch.Size([3, 10])


In [19]:
optimizer = optim.SGD(CNN.parameters(),lr=0.001,momentum=0.9)
loss_function = nn.CrossEntropyLoss()

lr_sche = optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma=0.5)

In [20]:
loss_plot = vis.line(Y=torch.Tensor(1).zero_())

In [23]:
epochs = 30
print("length of train_loader : {}".format(len(train_loader)))
for epoch in range(epochs):
    running_loss = 0.0
    check_loss = 0.0
    lr_sche.step()
    for num, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad() # this is really important thing!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        out = CNN(inputs)
        
        loss = loss_function(out, labels)
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        check_loss += loss.item()
        
        if (num % 100 == 1) and (num != 1):
            print("loss : {}".format(check_loss/100))
            plt_tracker(loss_plot,(check_loss), epoch *len(train_loader) + num)
            check_loss = 0            

        if num % len(train_loader) == (len(train_loader) - 1):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, num + 1, running_loss / len(train_loader)))
            
            running_loss = 0.0

length of train_loader : 6250


loss : 0.5467007020115853


loss : 0.5811240023374558


loss : 0.5533620603382587


loss : 0.6342594990134239


...

loss : 0.6010310932993889


loss : 0.5986859938502311


loss : 0.5795916929841042


loss : 0.5948571959137916


KeyboardInterrupt: 